___

# GridSearch: PyGSP for Mouse Phenotype prediction

In [28]:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,accuracy_score
from pygsp import graphs, filters, plotting, learning
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from tabulate import tabulate
from scipy.stats import norm
import networkx as nx
import pandas as pd
import numpy as np
import scipy
import os
import re

## Import Data

<div class="alert alert-block alert-info">
    
* genotype_df: contains the genotype of each strain for each allele
* phenotype_df : contains the phenotypes of each strain, lots of Nan

NB: on this branch of the project, we don't use the expression of the genes, and try instead to predict straight the phenotype from the genotype: therefore, we don't import any file from expression data.
</div>

In [34]:
genotype_df = pd.read_csv("data/genotype_BXD.txt", sep='\t', index_col='SNP')

# Remove F0 and F1 generation (parental)
genotype_df.drop(['B6D2F1', 'D2B6F1', 'C57BL.6J', 'DBA.2J'], axis=1, inplace=True)

genotype_df = genotype_df.transpose()
genotype_df.index.name = 'BXD_strain'

print("There is NaN values: %s" % genotype_df.isna().any().any())

phenotype_df = pd.read_csv("data/Phenotype.txt", sep='\t', index_col='PhenoID')

# Remove F0 and F1 generation (parental)
phenotype_df.drop(['B6D2F1', 'D2B6F1', 'C57BL.6J', 'DBA.2J'], axis=1, inplace=True)

nan_count = phenotype_df.isna().sum().sum()
entries_count = phenotype_df.shape[0] * phenotype_df.shape[1]

print("Number of Nan values: %s" % nan_count)
print("Percentage of nan in the phenotype file: {:0.2f}%".format(nan_count / entries_count * 100))

There is NaN values: False
Number of Nan values: 291981
Percentage of nan in the phenotype file: 61.66%


### Phenotype_id_aligner

In [35]:
# Phenotype id aligne is an in-depth description of each phenotype
phenotype_id_aligner = pd.read_csv('data/phenotypes_id_aligner.txt', sep='\t', encoding='latin1')
phenotype_id_aligner.rename({'Quantitive.trait': 'quantitative_trait'}, axis=1, inplace=True)
quant_pheno_best = phenotype_id_aligner[(phenotype_id_aligner.Strains > 70) & (phenotype_id_aligner.quantitative_trait == 'No')]
quant_pheno_best = quant_pheno_best.sort_values(by='Strains', ascending=False)

#### Weighted edges

In [38]:
def epsilon_similarity_graph(X: np.ndarray, metric='euclidean', sigma=1, epsilon=0):
    """ X (n x d): coordinates of the n data points in R^d.
        sigma (float): width of the kernel
        epsilon (float): threshold
        Return:
        adjacency (n x n ndarray): adjacency matrix of the graph.
    """
    Dists = squareform(pdist(X,metric = "euclidean"))
    Dists = np.exp(-Dists**2/(2 * sigma**2))
    Dists[Dists <= epsilon] = 0
    np.fill_diagonal(Dists,0)
    return Dists

# Exploitation

TODO:
* Throw a big gridsearch on many parameters
* adapt the grid_search_graphs function s.t. it can compute the adjacency matrix with the gene expression data.

<div class="alert alert-block alert-info">
At that point, all we need is a pygsp graph describing the different strains: the way it is linked is therefore critical for the success of the algorithm.
</div>

In [224]:
#small parameters for tests
METRICS = ['cosine']
SIGMAS = [60,70]
EPSILONS = [0.28]
TAUS = [0.1]

In [242]:
#real parameters
METRICS = ['cosine','jaccard','hamming','euclidian']
SIGMAS = np.arange(60,76,4)
EPSILONS = np.arange(0.2,0.4,0.02)
TAUS = [0,0.1,1,10]

In [237]:
def get_min_max(params_scores):
    ind_max = np.nanargmax([i[0] for i in params_scores])
    ind_min = np.nanargmin([i[0] for i in params_scores])
    maxo = params_scores[ind_max][0]
    mino = params_scores[ind_min][0]
    return mino,maxo

In [238]:
def grid_search_graphs(CV_meth,metrics,sigmas,epsilons,taus,gene):
    """
    CV_meth: a method performing cross validation (on discrete or continuous variable)
    metrics: list of metrics usable in a pdist matrix
    sigmas,epsilons: parameter for the adjacency matrix we do the grid search on
    """
    params_scores = []
    tot_comb = len(metrics) * len(sigmas) * len(epsilons) * len(taus)
    for i,met in enumerate(metrics):
        for j,sig in enumerate(sigmas):
            for k,eps in enumerate(epsilons):
                for l,tau in enumerate(taus):
                    print(" %i / %i combination" % (l + len(taus) * (k + len(epsilons) * (j + len(sigmas) * (i))),tot_comb))
                    try:
                        weighted_adjacency = epsilon_similarity_graph(genotype_df.values, metric=met, sigma=sig, epsilon=eps)
                        pygsp_weighted_graph = graphs.Graph(weighted_adjacency,lap_type='normalized') 
                        res = CV_meth(pygsp_weighted_graph,phenotype_df,genotype_df,tau,gene)
                        params_scores.append([res,(met,sig,eps,tau)])
                    except nx.NetworkXError:
                        print('non connected graph with %.2f sigma and %.2f epsilon' % (sig,eps))
                        continue
    mino,maxo = get_min_max(params_scores)
    print('#' * 32)
    print('Biggest result for gene %s : %.2f' % (gene,maxo))
    print('Smallest result for gene %s : %.2f' % (gene,mino))
    print('#' * 32)
    return params_scores
    

### Labels (hait coat color)

In [190]:
def CV_hair_label(pygsp_weighted_graph,phenotype_df,genotype_df,tau,gene = 'X62',NUM_REPET = 50,plot = False):
    y_true = phenotype_df.loc[gene][genotype_df.index].fillna(0)
    acc = 0
    for i in range(NUM_REPET):
        rs = np.random.RandomState()
        mask = rs.uniform(0,1,pygsp_weighted_graph.N) > 0.2
        labels_true = y_true[~mask]
        y_sparse = y_true.copy()
        y_sparse[~mask] = np.nan

        recovery = learning.classification_tikhonov(pygsp_weighted_graph,y_sparse,mask,tau=tau)
        #we get rid of the 0 value: we don't accept no label

        #recovery[:,0] = -1
        prediction = np.argmax(recovery, axis=1)

        labels_pred = prediction[~mask]
        acc += accuracy_score(labels_true,labels_pred)
    acc_final = acc/NUM_REPET
    if plot:
        print('Accuracy:',acc_final)
        fig, ax = plt.subplots(1, 3, sharey=True, figsize=(20, 6))
        #ground truth
        pygsp_weighted_graph.plot_signal(y_true, ax=ax[0], title='Ground truth')
        #measurements
        pygsp_weighted_graph.plot_signal(y_sparse, ax=ax[1], title='Measurements')
        #recover
        pygsp_weighted_graph.plot_signal(prediction, ax=ax[2], title='Recovered class')
    return acc_final

In [249]:
list_genes = ['X62']
dic_disc = {}
for gene in list_genes:
    result = grid_search_graphs(CV_hair_label,METRICS,SIGMAS,EPSILONS,TAUS,gene)
    dic_disc[gene] = result

 0 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 1 / 640 combination
 2 / 640 combination
 3 / 640 combination
 4 / 640 combination
 5 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 6 / 640 combination
 7 / 640 combination
 8 / 640 combination
 9 / 640 combination


...

 630 / 640 combination
 631 / 640 combination
 632 / 640 combination
 633 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 634 / 640 combination
 635 / 640 combination
 636 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 637 / 640 combination
 638 / 640 combination
 639 / 640 combination
################################
Biggest result for gene X62 : 0.53
Smallest result for gene X62 : 0.02
################################


### Continuous variable

As we can see, there are more than 10 continuous phenotype who have interesting charateristics as continuous features.

In [200]:
def CV_cont_phen(pygsp_weighted_graph,phenotype_df,genotype_df,tau,gene,NUM_REPET = 50,metric_meth = r2_score,plot = False):
    ycont_true = phenotype_df.loc[gene][genotype_df.index]
    ycont_true.fillna(ycont_true.mean(),inplace = True)
    #mae or mse
    mxe = 0
    for i in range(NUM_REPET):
        mask = np.random.RandomState().uniform(0,1,pygsp_weighted_graph.N) > 0.2
        vals_true = ycont_true[~mask]
        ycont_sparse = ycont_true.copy()
        ycont_sparse[~mask] = np.nan

        recovery = learning.regression_tikhonov(pygsp_weighted_graph,ycont_sparse,mask,tau=tau)
        if any(np.isnan(recovery)):
            return np.nan
        vals_pred = recovery[~mask]
        mxe += metric_meth(vals_true,vals_pred)

    mxe_avg = mxe / NUM_REPET
    if plot:
        print(str(metric_meth),mxe_avg)
        fig, ax = plt.subplots(1, 3, sharey=True, figsize=(20, 6))
        limits = [ycont_true.min(),ycont_true.max()]

        #ground truth
        pygsp_weighted_graph.plot(ycont_true, ax=ax[0],limits = limits, title='Ground truth')
        #measurements
        pygsp_weighted_graph.plot_signal(ycont_sparse, ax=ax[1],limits = limits, title='Measurements')
        #recover
        pygsp_weighted_graph.plot_signal(recovery, ax=ax[2],limits = limits, title='Recovered class')
    return mxe_avg

In [151]:
phenotype_df['nan_count'] = phenotype_df.isnull().sum(axis = 1)
top_continuous = phenotype_df.join(phenotype_id_aligner.set_index('PhenoID')[['quantitative_trait']],how = 'inner')
top_continuous = top_continuous[top_continuous.quantitative_trait == 'Yes'].sort_values('nan_count').head(5)
print('The best continuous variable are : {}'.format(' '.join(top_continuous.index.values)))

The best continuous variable are : X3820 X2397 X1002 X328 X218


In [243]:
list_cont_genes = ['X3820','X2397','X1002','X328','X218']
dic_cont = {}
for gene in list_cont_genes:
    results_cont = grid_search_graphs(CV_cont_phen,METRICS,SIGMAS,EPSILONS,TAUS,gene)
    ind_max = np.nanargmax([i[0] for i in results])
    r2 = results[ind_max]
    dic_cont[gene] = results_cont

 0 / 640 combination
 1 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 2 / 640 combination
 3 / 640 combination
 4 / 640 combination
 5 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 6 / 640 combination
 7 / 640 combination
 8 / 640 combination
 9 / 640 combination
 10 / 640 combination


...

 630 / 640 combination
 631 / 640 combination
 632 / 640 combination
 633 / 640 combination
 634 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 635 / 640 combination
 636 / 640 combination
 637 / 640 combination


/home/ymentha/anaconda3/envs/ntds_2019/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:193: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


 638 / 640 combination
 639 / 640 combination
################################
Biggest result for gene X218 : -0.29
Smallest result for gene X218 : -564.41
################################


In [248]:
get_min_max(dic_cont['X218'])

(-564.4125417244661, -0.2909246803554421)